In [14]:
import gradio as gr
from PIL import Image
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import io, os, csv, uuid
 
 
# Azure Custom Vision API setup
prediction_key = "4XfrK7iNUEKB3hhvtbHCibpwR5jy5JJWKiPXm7nPSgiH8Kx4wvVmJQQJ99AKACmepeSXJ3w3AAAIACOGfQYj"
endpoint = "https://closetmate-prediction.cognitiveservices.azure.com/"
project_id = "fb78e869-3df8-4374-b6d5-2587fffbf22e"
iteration_name = "Iteration2"
 
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=endpoint, credentials=credentials)
 
# Define a dictionary for category translations
category_translation = {
    "KN": "니트", "JP": "자켓", "BL": "블라우스",
    "BG": "가방", "SH": "신발", "HT": "모자",
    "PT": "바지", "OP": "원피스", "SE": "셋업",
    "SC": "스카프", "CT": "코트", "JK": "점퍼",
    "SW": "스웨터", "CD": "카디건", "SK": "스커트", "VT": "조끼"
}
 
 
# 카테고리 필터를 위한 선택지 리스트
category_filter_choices = ["전체"] + list(category_translation.values())
 
# 옷장 정보를 저장할 파일 경로
wardrobe_file_path = "./data/wardrobe.csv"
 
# 옷 이미지 경로
image_dir = "./data/closet"
 
# 이미지를 예측하고 저장하는 함수
def predict_image(image, image_name, save_dir=image_dir):
    # 저장할 디렉토리가 없으면 새로 생성
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
 
    # 이미지를 리사이즈하고 예측을 위해 준비
    resized_image = image.convert("RGB").resize((224, 224))
   
    # 이미지를 로컬에 저장
    image_path = os.path.join(save_dir, image_name)
    resized_image.save(image_path)
 
    # 예측을 수행
    with io.BytesIO() as img_data:
        resized_image.save(img_data, format="PNG")
        img_data.seek(0)
        results = predictor.classify_image(project_id, iteration_name, img_data)
 
    # 예측 결과를 처리
    predictions = {
        category_translation.get(prediction.tag_name, prediction.tag_name): prediction.probability
        for prediction in results.predictions if prediction.probability > 0.2  # 확률이 20%보다 큰 예측만 필터링
    }
    # 예측 결과를 확률에 따라 내림차순으로 정렬
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    result_str = "\n".join([f"{tag}: {prob * 100:.2f}%" for tag, prob in sorted_predictions])
 
    # 예측 결과를 반환
    return result_str if result_str else "예측된 결과가 없습니다."
 
 
 
# 파일에서 옷장 정보 로드
def load_wardrobe():
    wardrobe = []
    if os.path.exists(wardrobe_file_path):
        with open(wardrobe_file_path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                wardrobe.append(row)
    return wardrobe
 
# 파일에 옷장 정보 저장
def save_wardrobe(wardrobe):
    with open(wardrobe_file_path, mode='w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["Image", "Category", "Confidence"])
        writer.writeheader()
        for item in wardrobe:
            writer.writerow(item)
 
# 이미지 분류 후 옷장에 추가하는 함수
def classify_and_add_to_wardrobe(image):
    # 고유한 파일 이름을 생성
    image_name = f"{uuid.uuid4()}.png"
 
    result = predict_image(image, image_name)
    if result and "예측된 결과가 없습니다" not in result:
        lines = result.split("\n")
        main_category = lines[0].split(":")[0]
 
        if main_category not in category_filter_choices:
            category_filter_choices.append(main_category)
 
        # 현재 옷장 정보를 로드
        wardrobe = load_wardrobe()
       
        # 이미지와 예측 정보를 옷장에 추가
        wardrobe.append({"Image": image_name, "Category": main_category, "Confidence": lines[0].split(": ")[1]})
       
        # 옷장 정보를 파일에 저장
        save_wardrobe(wardrobe)
   
    # 결과 문자열 반환
    return result
 
# 옷을 삭제하는 함수
def delete_item(image_name):
    wardrobe = load_wardrobe()
   
    # 삭제할 이미지와 일치하는 항목을 찾기
    item_to_delete = next((item for item in wardrobe if item["Image"] == image_name), None)
   
    if item_to_delete:
        # 이미지 파일 삭제
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            os.remove(image_path)
       
        # CSV에서 해당 항목 삭제
        wardrobe = [item for item in wardrobe if item["Image"] != image_name]
        save_wardrobe(wardrobe)
        return f"{image_name} 이미지가 삭제되었습니다."
    else:
        return f"해당 이미지가 옷장에 존재하지 않습니다."
   
 
# 옷장에 저장된 이미지를 경로와 함께 반환하도록 수정
def view_wardrobe(category_filter):
    wardrobe = load_wardrobe()
   
    if category_filter == "전체":
        filtered_db = wardrobe
    else:
        filtered_db = [item for item in wardrobe if item["Category"] == category_filter]
   
    # 옷장 정보에서 이미지 파일 이름과 함께 실제 이미지 경로를 반환
    items = [(os.path.join("./data/closet", item["Image"]), f"{item['Category']} ({item['Confidence']})") for item in filtered_db]
    return items
 
# Gradio Interface with CSS styling
with gr.Blocks(css=f"""
    html, body {{
        margin: 0 !important;
        padding: 0 !important;
        overflow-x: hidden;
        overflow-y: auto;
        width: 100vw !important;
        height: 100vh !important;
        background-image: url('https://i.ibb.co/D9PBSTT/5.png') !important;
        background-size: cover !important;
        background-repeat: no-repeat !important;
        background-position: center !important;
    }}    
    .gradio-container {{
        background-image: url('https://i.ibb.co/D9PBSTT/5.png') !important;
        background-size: cover !important;
        background-repeat: no-repeat !important;
        background-position: center !important;
        font-family: 'Arial', sans-serif;
        width: 100vw !important;
        max-width: 100vw !important;
        height: 100vh !important;
        overflow-y: auto;
    }}              
               
    .gr-box, .gr-block, .gr-column, .gr-row, .gr-tab-item {{
        background-color: rgba(255, 255, 255, 0.8) !important;
        border-radius: 12px;
        border: 1px solid #FFD1D1;
        width: 100% !important;
        box-sizing: border-box;
    }}
    .gr-button {{
        background-color: #FFE4E1 !important;
        color: white !important;
        font-weight: bold;
        border-radius: 8px;
        padding: 8px 16px;
    }}
    .gr-button:hover {{
        background-color: #D81B60 !important;
    }}
    .gr-tabs .gr-tab-item {{
        font-weight: bold;
        color: #880E4F !important;
    }}
 
    .gallery-image {{
        width: 250px !important;
        height: 250px !important;
    }}
               
    .svelte-633qhp {{
        background: transparent; /* 배경을 투명으로 변경 */
        border: none; /* 테두리 제거 */
    }}
 
    .svelte-7ddecg {{
        background-color: rgba(255, 192, 203, 0.8); /* 원하는 배경색 (예: 연한 분홍색) */
        padding: 5px 10px; /* 텍스트와 배경 사이에 여백 추가 */
        border-radius: 5px; /* 둥근 모서리 효과 */
        display: inline-block; /* 텍스트 길이에 맞춰 배경이 조절됨 */
    }}
 
    /* TabItem 버튼의 스타일 지정 */
    .svelte-1tcem6n {{
        background-color: rgba(255, 240, 245, 0.8); /* 배경색 추가 */
        border-radius: 8px; /* 버튼 모서리를 둥글게 */
        padding: 8px 16px;
    }}
    .svelte-1tcem6n.selected {{
        background-color: rgba(216, 27, 96, 0.8); /* 선택된 탭의 배경색 */
        color: white; /* 글자색 변경 */
        font-weight: bold;
    }}          
       .narrow-element {{
        max-width: 300px !important;
        height: auto !important;
        margin: 0 auto !important;
    }}
    .image-frame img {{
        width: 298px; /* 원하는 고정 너비 */
        height: 240px; /* 원하는 고정 높이 */
    }}
 
    .gradio-container.gradio-container-5-5-0 .contain img.svelte-1pijsyv {{
        object-fit: contain;
    }}
       
 
""") as app:
 
    with gr.TabItem("의류 추가"):
        gr.Markdown("### 👗 내옷짱 : 의류 관리 서비스")
        image_input = gr.Image(type="pil", label="의류 이미지 업로드", elem_classes="narrow-element")
        result_output = gr.Textbox(label="분류 결과", elem_classes="narrow-element")
        add_button = gr.Button("분류 및 추가", elem_classes="narrow-element")
       
        # Update both result_output and category filter choices
        add_button.click(
            fn=classify_and_add_to_wardrobe,
            inputs=image_input,
            outputs=result_output
        )
 
    with gr.TabItem("옷장 조회") as wardrobe_tab:
        gr.Markdown("### 🛍️ 옷장 조회")
        category_filter = gr.Dropdown(label="카테고리 필터", choices=category_filter_choices, value="전체")
        wardrobe_gallery = gr.Gallery(label="옷장 보기")
       
        # Dropdown이 변경될 때 바로 view_wardrobe 함수 실행
        category_filter.change(fn=view_wardrobe, inputs=category_filter, outputs=wardrobe_gallery)
        # 탭이 열릴 때 '전체'로 조회
        wardrobe_tab.select(fn=view_wardrobe, inputs=category_filter, outputs=wardrobe_gallery)
 
 
       
 
app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://7f4b0d3e3f7dd1e8fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
pip install azure-cognitiveservices-vision-customvision


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install msrest


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
